In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
import pandas_datareader.data as web
import datetime as dt
import matplotlib.dates as mdates

In [3]:
# 构造数据源URL
server = "http://quotes.money.163.com/"
path = "service/"
table1 = "zcfzb_"
table2 = "lrb_"
table3 = "xjllb_"
stock = "600585"
type = ".html?type=year"

target1 = server + path + table1 + stock + type
target2 = server + path + table2 + stock + type
target3 = server + path + table3 + stock + type

In [4]:
# 获取三张财务报表原始数据
df1 = pd.read_csv(target1, header=0, prefix="V", encoding="gb18030")
df2 = pd.read_csv(target2, header=0, prefix="V", encoding="gb18030")
df3 = pd.read_csv(target3, header=0, prefix="V", encoding="gb18030")

In [5]:
# 报表行列转置
bs = df1.transpose()
pl = pd.DataFrame(df2.values.T, columns=df2['报告日期'], index=df2.columns)
cf = pd.DataFrame(df3.values.T, columns=df3[' 报告日期'], index=df2.columns)

In [6]:
# 将字符型数据转换成数值型
headers_bs = bs.columns
headers_pl = pl.columns
headers_cf = cf.columns


for head in headers_pl:
    pl[head] = pd.to_numeric(pl[head], errors='coerce').fillna(0) # 不符合转换条件的设为‘Nan’，再用0替换

for head in headers_pl:
    pl[head] = pd.to_numeric(pl[head], errors='coerce').fillna(0)

for head in headers_cf:
    cf[head] = pd.to_numeric(cf[head], errors='coerce').fillna(0)

In [7]:
pl_sum = pd.DataFrame()

pl_sum['total_revenu'] = pl['营业总收入(万元)']
pl_sum['growth_rate'] = (pl['营业总收入(万元)'] - pl['营业总收入(万元)'].shift(-1)) / pl['营业总收入(万元)']
pl_sum['intre_expen'] = pl['利息支出(万元)']
pl_sum['total_profit'] = pl['利润总额(万元)']
pl_sum['income_tax'] = pl['所得税费用(万元)']
pl_sum['tax_rate'] = pl['所得税费用(万元)'] / pl['利润总额(万元)']

pl_sum.drop('报告日期', inplace=True)    # 删除第一行
pl_sum.drop(pl_sum.tail(1).index, inplace=True)    # 删除最后一行

In [8]:
pl_sum

,total_revenu,growth_rate,intre_expen,total_profit,income_tax,tax_rate
2021-12-31,16795266.0,-0.049359,0.0,4411620.0,995036.0,0.225549
2020-12-31,17624268.0,0.109011,0.0,4710792.0,1073774.0,0.227939
2019-12-31,15703033.0,0.182307,0.0,4455685.0,1020484.0,0.229030
2018-12-31,12840263.0,0.413479,0.0,3962920.0,899318.0,0.226933
2017-12-31,7531082.0,0.257319,0.0,2122876.0,480002.0,0.226109
2016-12-31,5593190.0,0.088605,0.0,1165321.0,270256.0,0.231915
2015-12-31,5097604.0,-0.191903,0.0,1003940.0,241145.0,0.240199
2014-12-31,6075850.0,0.090470,0.0,1488281.0,329521.0,0.221410
2013-12-31,5526168.0,0.171828,0.0,1263127.0,281981.0,0.223240
2012-12-31,4576620.0,-0.063095,0.0,808782.0,162559.0,0.200992


In [9]:
cf_sum = pd.DataFrame()

cf_sum['depr_fixed'] = cf[' 固定资产折旧、油气资产折耗、生产性物资折旧(万元)']    # 注意column的标题有一个前置空格
cf_sum['intang_amort'] = cf[' 无形资产摊销(万元)']
cf_sum['deferred'] = cf[' 长期待摊费用摊销(万元)']
cf_sum['depreciation'] = cf[' 固定资产折旧、油气资产折耗、生产性物资折旧(万元)']+cf[' 无形资产摊销(万元)']+cf[' 长期待摊费用摊销(万元)']

cf_sum.drop('报告日期', inplace=True)    # 删除第一行
cf_sum.drop(cf_sum.tail(1).index, inplace=True)    # 删除最后一行

In [10]:
cf_sum

,depr_fixed,intang_amort,deferred,depreciation
2021-12-31,535191.0,60485.0,0.0,595676.0
2020-12-31,498444.0,44512.0,0.0,542956.0
2019-12-31,493163.0,33934.0,0.0,527097.0
2018-12-31,464730.0,30611.0,0.0,495341.0
2017-12-31,459028.0,24290.0,0.0,483318.0
2016-12-31,449763.0,25508.0,0.0,475271.0
2015-12-31,421212.0,24720.0,0.0,445932.0
2014-12-31,359924.0,22853.0,0.0,382777.0
2013-12-31,338623.0,18783.0,0.0,357406.0
2012-12-31,311542.0,16052.0,0.0,327594.0


In [11]:
valuation = pd.DataFrame()